# code结构
    + 读取txt,建立一个mappoint类
    + 读取整个rgb文件夹，用一个列表维护所有的图像文件名,列表mats
    + 读取整个txt，对里面的结果进行解析，并不断的新建mappoint,所有的mappoint用一个points列表维护
    + 遍历points列表，对于每一个point,求解它的class_id,这里很费时间的，赶紧把类别搞清楚，instance先不管了
    + 这份代码没有考虑instance,所以要更新一份别的了，emmm，得到points结果后写到txt，把class_id解析进去就好了

In [2]:
class MapPoint():
    def __init__(self, global_id):
        self.global_id = global_id
        self.mat_id = -1
        self.x_corrdi = -1
        self.y_corrdi = -1
        self.class_id = -1
        self.class_name = 'Background 100%'

In [3]:
import os

path = 'rgb/'
dirs = os.listdir(path)
mats = sorted(dirs, key = lambda x: float(x[:-4]))
#dirs = sorted([float(x[:-4]) for x in dirs])
#mats = [str(x)+'.png' for x in dirs]


In [4]:
cnt = 0
points = []
with open("scmantic_saving.txt", "r") as f:
    for line in f: # for line in f 竟然就是逐行读去了，醉了醉了
        s = line.strip('\n').split(' ')
        #print(s)
        if s[0] == 'global':
            point = MapPoint(int(s[3]))
            point.mat_id = int(s[4])
            point.x_corrdi = float(s[6])
            point.y_corrdi = float(s[7])
            points.append(point)
        else:
            pass
        #cnt += 1
        #if cnt > 10:
        #    break

In [5]:
import detectron2
import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog

cfg = get_cfg()
cfg.merge_from_file("/home/zherlock/InstanceDetection/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")   # load values from a file
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.merge_from_list(["MODEL.WEIGHTS", "/home/zherlock/InstanceDetection/detectron2/pre_train_model/model_final_f10217.pkl","MODEL.DEVICE","cpu"]) 
#print(cfg.dump())
predictor = DefaultPredictor(cfg)

metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])


In [6]:
def _create_text_labels(classes, scores, class_names):
    """
    Args:
        classes (list[int] or None):
        scores (list[float] or None):
        class_names (list[str] or None):

    Returns:
        list[str] or None
    """
    labels = None
    if classes is not None and class_names is not None and len(class_names) > 1:
        labels = [class_names[i] for i in classes]
    if scores is not None:
        if labels is None:
            labels = ["{:.0f}%".format(s * 100) for s in scores]
        else:
            labels = ["{} {:.0f}%".format(l, s * 100) for l, s in zip(labels, scores)]
    return labels

In [7]:
#挺多地图点都出现在同一个关键帧里，用一个hashmap存储一个图片的结果，如果遇到相同的就查找就好
#并不是每个点都属于物体呢,挺多关键点在背景里面的

In [8]:
#图片是640*480的，可能是行列弄反了吧
#len(output["instances"].pred_masks[0] is 480
#len(output["instances"].pred_masks[0][0] is 640

In [14]:
import time

#字典，用于保存一个图片识别的结果
seg_output = {}
#字典，保存一个图片id下的
mat_dir = os.path.join(os.getcwd(), 'rgb')
cnt = 0
recognize = 0
start = time.time()
for point in points:
    print(point.global_id)
    mat_path = os.path.join(mat_dir, mats[point.mat_id])
    if point.mat_id in seg_output:
        output = seg_output[point.mat_id]
        if point.global_id == 3:
            print('labels are', output['labels'])
            print('len of labels is', len(output['labels']))
            print('socres are', output["instances"].scores)
        for i in range(len(output["instances"])):
            if output["instances"].pred_masks[i][int(point.y_corrdi)][int(point.x_corrdi)]:
                point.class_id = output["instances"].pred_classes[i]
                point.class_name = output['labels'][i]
                print('point {} belongs to {}'.format(point.global_id, point.class_name))
                recognize += 1
                break #假设不存在一个像素被两个instance占用
    else:
        im = cv2.imread(mat_path)
        #cv2.imshow('test',im)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #break
        output = predictor(im)
        #print('x coor is ',len(output["instances"].pred_masks[0]))
        #print('y coor is ',len(output["instances"].pred_masks[0][0]))
        #break
        output['labels'] = _create_text_labels(output["instances"].pred_classes, output["instances"].scores, metadata.get("thing_classes", None))
        seg_output[point.mat_id] = output
        for i in range(len(output["instances"])):
            if output["instances"].pred_masks[i][int(point.y_corrdi)][int(point.x_corrdi)]:
                point.class_id = output["instances"].pred_classes[i]
                point.class_name = output['labels'][i]
                print('point {} belongs to {}'.format(point.global_id, point.class_name))
                recognize += 1
                break
    #cv2.imshow('test',im)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #print(mat_path)
    cnt += 1
    if cnt > 10:
        break
print("{} points are recognized ".format(recognize))
end = time.time()
print('{} seconds has past'.format(end - start))

0
1
2
3
labels are ['tv 77%', 'laptop 57%', 'book 56%', 'laptop 52%']
len of labels is 4
socres are tensor([0.7711, 0.5690, 0.5596, 0.5242])
point 3 belongs to laptop 52%
4
5
6
7
point 7 belongs to laptop 52%
8
9
10
2 points are recognized 
4.7809367179870605 seconds has past


In [ ]:
#1589 points are recognized 
#389.0116274356842 seconds has past,宝贵的６分钟噢


In [ ]:
float('1.000')

In [ ]:
import time
start = time.time()
time.sleep(5)
end = time.time()
print('{} seconds has past'.format(end - start))

In [ ]:
float('1305031452.791720')

In [ ]:
mats[0]

In [ ]:
points[3].global_id

In [ ]:
points[5].class_name

In [ ]:
os.path.join(os.getcwd(), 'rgb')

In [ ]:
len(points)

In [ ]:
points[0].mat_id

In [ ]:
os.getcwd()

In [ ]:
mats[34]

In [ ]:
points[0].global_id

In [ ]:
with open("scmantic_saving.txt", "r") as f:
    while f.readline():
        print(f.readline())
        break

In [ ]:
mats[0]

In [ ]:
mats[-1]

In [ ]:
a = [1,2,3,4]
a[:-1]

In [ ]:
dirs[0]

In [ ]:
dirs[1]

In [ ]:
dirs[2]

In [ ]:
dirs[3]

In [ ]:
ma = MapPoint(3)
ma.id

In [ ]:
ma.mat_Id = 5
ma.mat_Id